## This file will navigate to a directory with multiple CSVs and compile ALL of them into 1 master CSV, with the header

In [1]:
# import dependencies
import pandas as pd
import os
import glob

In [61]:
# path = 'Data/Geocoded/'
path_files = glob.glob(os.path.join("*.csv"))
df = pd.concat((pd.read_csv(f) for f in path_files)).drop_duplicates().reset_index(drop=True)

# specify the columns to drop based on their 0-based index
columns_to_drop = [
    4, 
    6, 
    7, 
    10, 
    11
]

# now, drop!
df.drop(
    df.columns[columns_to_drop], 
    axis=1, 
    inplace=True
)

# rename columns
new_column_names = {
    'Sale Date': 'sale_date',
    'Sale Price': 'sale_price',
    'Year  Built ':'yr_built',
    'Square Ft ': 'square_feet',
}

df.rename(
    columns=new_column_names, 
    inplace=True
)

# change sale_price column to a float
df['sale_price'] = df['sale_price'].str.replace(',', '').str.replace('$', '').astype(float)


# create necessary additional columns
df['full_address'] = df['Address'].str.title() + ' Rockdale County GA'
df['unique_ID'] = df['Parcel ID'] + '-' + df['sale_date'] + '-' + df['sale_price'].astype(str)
df['price_sf'] = df['sale_price'] / df['square_feet']

# this step removes multi-parcel transactions
df = df[df['Reason'] != 90]

# Now that we've used the 'Reason' column, drop it
df.drop(
    'Reason', 
    axis=1, 
    inplace=True
)

# compare the yr_blt value to when the property sold. Want to weed out all sales with a home constructed after the year of sale
df['sale_date'] = pd.to_datetime(df['sale_date'])
df['sale_year'] = df['sale_date'].dt.year
df = df[df['yr_built'] <= df['sale_year']]

# create the columns that will be used by the chart
df['year'] = pd.DatetimeIndex(df['sale_date']).year
df['month'] = pd.DatetimeIndex(df['sale_date']).month
df['year-month'] = df['year'].astype(str) + '-' + df['month'].astype(str)

# drop rows with ridiculously small structures
df = df[df['square_feet'] >=500]

# export dataframe to CSV to be geocoded
df.to_csv('../Geocode/Rockdale_ready4geocoder.csv')

print('export complete!')

export complete!
